# City Networks - Full example

This notebook presents a full example of constructing, analysing and visualising city data from the CERL Thesaurus -- starting from getting the data out of the database to plotting it on a HTML map that can easily be shared.

Author: Andreas Lüschow

Last updated: 2021/09/20

-----

## Part 1 - Querying the database

First of all, we have to import some python modules and packages to work with our data.

In [1]:
import json  # for reading/writing JSON data
import os  # for file system manipulation (i.e., creating files etc.)

from cerl import ample_query, ample_record, ids_from_result, by_dot, the  # CERL specific python library, see https://pypi.org/project/cerl/

At the beginning of each python script, it's always a good idea to define variables that are used throughout the script. By this, we are able to find and change our parameters easily.

In [2]:
# the file path
DOWNLOAD_PATH = "./data/ct.json"  # data from the CERL Thesaurus will be downloaded into this file

# create the directory in the download path if it does not already exist
if not os.path.exists(os.path.dirname(DOWNLOAD_PATH)):
    os.makedirs(os.path.dirname(DOWNLOAD_PATH))
    
# the output data    
download_data = {}  # a python dictionary that will contain the data we collect    

# the database used
DATABASE = 'data.cerl.org/thesaurus'

# the search query used
QUERY = 'related_to:cnl00029316 AND type:cnp'  # i.e., get people records related to Göttingen

Next, we have to make a connection to the database, send a search query and handle the retrieved data sets afterwards. This may take a minute or two.

__As the search query defined above shows, we are interested in people that were active in Göttingen. From these person records we will collect all information about places of activity. We will then construct a network of cities.__

In [ ]:
# Connect to the CERL Thesaurus and run the search query
result = ample_query(DATABASE, QUERY)

In [ ]:
# iterate over the search results
for idx in ids_from_result(result):
    record = ample_record(DATABASE, idx)  # get the record as a python dictionary
    cid = the(by_dot(record, '_id'))  # access the record by dot notation (see https://pypi.org/project/cerl/ for more explanation)
    assert cid == idx  # just to make sure the correct record was downloaded

    download_data[cid] = {"515": {"a": []}}  # add information about the record ID, the field and subfield to the output dictionary

    # iterate over places in the record
    for places in by_dot(record, "data.place"):
        for place in places:
            try:
                type_place = place["typeOfPlace"]  # try to get type of place
                place_id = place["id"]  # try to get place identifier
                
                # add combination of place type and ID to output dictionary
                if type_place == "actv" and place_id is not None:
                    download_data[cid]["515"]["a"].append(place_id)
            except:
                pass        

    # if no place of activity was found, remove empty entries from the output dictionary
    if len(download_data[cid]["515"]["a"]) == 0:
        del download_data[cid]

We can now save the data to a JSON file.

In [ ]:
with open(DOWNLOAD_PATH, "w", encoding="utf-8") as f:
    json.dump(download_data, f, indent=4)

Let's have a look at a part of the data we just created.

In [ ]:
with open(DOWNLOAD_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)
    
# show five entries from the data
for x in list(data)[:5]:
    print(x, data[x])

-----

## Part 2 - Creating an edge list

Next, we create a graph representation of our data, i.e., a format were nodes and edges are defined. We use the python library _Bibliometa_ to this end.

In [ ]:
import pandas as pd  # python library for handling tabular data

from bibliometa.graph.conversion import JSON2EdgeList
from bibliometa.graph.similarity import Similarity

Let's define some paths were our input and output data will be located.

In [ ]:
SIMILARITY_PATH = "./data/similarity.csv"
GRAPH_CORPUS_PATH = "./data/graph_corpus.json"
LOG_PATH = "./data/logs/j2e.log"

The Bibliometa class `JSON2EdgeList` needs some configuration to work properly.

In [ ]:
# fields from the input data that will be considered
FIELDS = [("515", "a")]

# whether keys and values from the input data will be switched
# since we downloaded person records but want to create a network of places, we set this parameter to True
SWAP = True  

# Similarity functions that will be used in graph creation.
# Two nodes (i.e., cities) are only considered connected if their similarity is greater then zero.
# See the Bibliometa documentation for more information about this.
# The following similarity function will consider two cities connected, if they occur together in at least one person record.
SIM_FUNCTIONS = [
    {"name": "mint_1",
     "function": Similarity.Functions.mint,
     "args": {
         "f": lambda a, b: len(list(a.intersection(b))),
         "t": 1}
     }
]

We can now run start the JSON to edge list conversion.

In [ ]:
j2e = JSON2EdgeList()

j2e.set_config(i=DOWNLOAD_PATH,
               o=SIMILARITY_PATH,
               create_corpus=True,
               corpus=GRAPH_CORPUS_PATH,
               log=LOG_PATH,
               fields=FIELDS,
               sim_functions=SIM_FUNCTIONS,
               swap=SWAP
               )
j2e.start()

The edge list looks as follows. Column 1 and 2 contain the name of a node (i.e., place), column 3 contains the number of connections between these nodes.

In [ ]:
with open(SIMILARITY_PATH, "r", encoding="utf-8") as f:
    df = pd.read_csv(f, sep="\t", header=None, index_col=0)
df[:10]

-----

## Part 3 - Graph Analysis

Based on the edge list we can run some graph analysis algorithms. This will also create a graph representation in GraphML format which we will user later on.

In [ ]:
# imports
from bibliometa.graph.analysis import GraphAnalysis

In [ ]:
SIMILARITY_FILE = "./data/similarity.tar.gz"
OUTPUT_FILE = "./data/graph_analysis.txt"
IMG_FOLDER = "./data/img/"
LOG_PATH = "./data/logs/ga.log"

CREATE_GRAPHML = True
GRAPHML_FILE = "./data/graphml.graphml"

NODES = "cities"  # name for nodes
EDGES = "similarity"  # name for edges
SIMILARITY_FUNCTION = "mint_1"  # name of similarity function used in graph creation
SIMILARITY_FUNCTIONS_ALL = ["mint_1"]  # list of available similarity functions (see previous conversion step from JSON to edge list)
THRESHOLD = 0  # threshold of similarity function
WEIGHTED = True  # whether similarity function is weighted

# analyses that will be used
TYPES = ["node_count",
         "edge_count",
         "component_count",
         "max_component"]

Run the analysis.

In [ ]:
ga = GraphAnalysis()

ga.set_config(i=SIMILARITY_FILE,
              o=OUTPUT_FILE,
              create_graphml=CREATE_GRAPHML,
              graphml=GRAPHML_FILE,
              img=IMG_FOLDER,
              log=LOG_PATH,
              n=NODES,
              e=EDGES,
              sim=SIMILARITY_FUNCTION,
              sim_functions=SIMILARITY_FUNCTIONS_ALL,
              t=THRESHOLD,
              weighted=WEIGHTED,
              types=TYPES
              )
ga.start()

-----

## Part 4 - Network creation

With the GraphML file created in the previous step we now can draw a nice network of cities and their connections.


In [ ]:
import networkx as nx  # a library for network modeling and analysis
import osmnx as ox  # a library for more beautiful network visualisation

At this point, we have to decide for which city want to draw the network. Since we downloaded data sets related to Göttingen, it's reasonable to draw a network around Göttingen.

In [ ]:
COORDINATES = "./data/lng_lat.csv"  # path to a CSV file with longitude/latitude information 
KEYS_LABELS = ("id", "city")  # needed for parsing the coordinates file

ego = "cnl00029316"  # CERL Thesaurus ID for Göttingen 
EGO_DEPTH = 1  # degree of neighbor nodes considered in the network
EGO_FILE = "./data/ego.graphml"  # a network centered around the "ego" city
HTML_FILE = "./data/network.html"  # output HTML representation of the network

CRS = "epsg:4326"  # coordinate reference system

The following code is a bit longer than in the previous steps, so we split it into three parts.

In [ ]:
# function for getting position of graph nodes
def create_pos(graph, df, keys_labels):
    pos = {}
    for n in graph.nodes:
        lat = df[df[keys_labels] == n]["lat"].to_string(index=False)
        lng = df[df[keys_labels] == n]["lng"].to_string(index=False)
        try:
            pos[n] = [float(lng), float(lat)]
        except:
            pos[n] = [None, None]
    return pos

In [ ]:
# read GraphML file
g = nx.read_graphml(GRAPHML_FILE)

# create ego network around ego node
g = nx.ego_graph(g, ego, radius=EGO_DEPTH)

# read coordinates
df = pd.read_csv(COORDINATES, sep="\t")

# create positions for nodes in the ego network
pos = create_pos(g, df, KEYS_LABELS[0])

# remove nodes without reasonable coordinate information
for node, (x, y) in pos.items():
    if x and y:
        g.nodes[node]['x'] = float(x)
        g.nodes[node]['y'] = float(y)
    else:
        g.remove_node(node)

# save ego network as GraphML
nx.write_graphml(g, EGO_FILE)

In [ ]:
# create MultiDiGraph from ego network
G = nx.MultiDiGraph(crs=CRS)
for n in g.nodes:
    G.add_node(n, x=g.nodes[n]["x"], y=g.nodes[n]["y"])
G.add_edges_from(g.edges)

# plot ego network on map and save as HTML
m = ox.folium.plot_graph_folium(G, color='#fe7830', weight=1, opacity=0.7)
m.save(HTML_FILE)

-----